In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import fetch_20newsgroups
cats = ['rec.autos', 'rec.motorcycles',
        'sci.crypt', 'sci.electronics', 
        'sci.med', 'sci.space',
        'talk.politics.guns', 'talk.religion.misc',
        'rec.sport.baseball', 'rec.sport.hockey']
#cats = None
df = fetch_20newsgroups(subset = 'all', categories=cats)

from pprint import pprint
pprint(list(df.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [3]:
df.filenames.shape

(11314,)

In [4]:
df.target.shape

(11314,)

In [5]:
# Convert to list
data = df.data

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

<input>:5: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \s
<ipython-input-5-1eed9f39b362>:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-5-1eed9f39b362>:8: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [8]:
# NLTK Stop words
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aserg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# !python -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 's', 'thing', 'car', 'nntp_post', 'host', 'rac_wam', 'umd', 'organization', 'university', 'maryland_college', 'park', 'line', 'wonder', 'anyone', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'whatev', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 5), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1)]]


In [11]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('maryland_college', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_post', 1),
  ('organization', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('umd', 1),
  ('university', 1),
  ('whatev', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
pprint(lda_model.print_topics())

[(0,
  '0.153*"window" + 0.065*"image" + 0.045*"display" + 0.044*"server" + '
  '0.037*"screen" + 0.020*"directory" + 0.020*"run" + 0.020*"font" + '
  '0.017*"cub" + 0.016*"object"'),
 (1,
  '0.053*"armenian" + 0.041*"greek" + 0.033*"turk" + 0.027*"turkish" + '
  '0.021*"turkey" + 0.018*"russian" + 0.015*"gif" + 0.014*"serdar_argic" + '
  '0.013*"armenia" + 0.013*"closed"'),
 (2,
  '0.042*"not" + 0.030*"would" + 0.027*"do" + 0.021*"be" + 0.016*"get" + '
  '0.015*"go" + 0.015*"think" + 0.014*"make" + 0.014*"good" + 0.014*"know"'),
 (3,
  '0.095*"sale" + 0.043*"tape" + 0.042*"character" + 0.030*"price" + '
  '0.029*"sell" + 0.023*"chris" + 0.023*"offer" + 0.021*"rider" + '
  '0.017*"jason" + 0.015*"decent"'),
 (4,
  '0.069*"color" + 0.059*"test" + 0.058*"video" + 0.049*"monitor" + '
  '0.046*"scan" + 0.039*"mouse" + 0.029*"mhz" + 0.020*"mirror" + '
  '0.020*"battery" + 0.018*"convert"'),
 (5,
  '0.049*"bike" + 0.028*"eat" + 0.028*"kid" + 0.026*"air" + 0.021*"dod" + '
  '0.021*"doctor" + 

In [44]:
def average_precision_at_k(theta, labels, k, dist_fn):
    dist = dist_fn(theta)
    np.fill_diagonal(dist, np.inf)
    idx = np.argpartition(dist, k-1)[:k,:]
    return np.mean(labels[idx] == labels[None,:])

In [61]:
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances

def hellinger(theta):
    return euclidean_distances(np.sqrt(theta))

def dot(theta):
    return 1 - np.dot(theta, theta.transpose())

In [ ]:
from numba import jit, prange, autojit

In [ ]:
@jit(nopython = True, parallel = True)
def outer(X, fn):
    dist = np.empty((X.shape[0], X.shape[0]), dtype = np.float32)
    for i in prange(X.shape[0]):
        for j in prange(i):
            dist[j,i] = dist[i,j] = fn(X[i,:], X[j,:])           
    return dist

In [ ]:
@jit(nopython = True)
def sym_kl(p, q):
    return kl(p, q) + kl(q, p)

In [ ]:
@jit(nopython = True)
def js(p, q):
    m = (p + q) / 2
    return kl(p, m) / 2 + kl(q, m) / 2

In [ ]:
@jit(nopython = True)
def bc(p, q):
    return -np.log(np.sum(np.sqrt(p * q)))

In [63]:
dist_fns = {'cosine' : cosine_distances,
            'eucl' : euclidean_distances,
            'hellinger' : hellinger,
            'dot' : dot
           }

In [64]:
from gensim import matutils

def eval_topic_model(model, labels, k = 100, dist_fns = dist_fns):
    matrix = model.get_document_topics(corpus, minimum_probability=0)
    theta = matutils.corpus2dense(matrix, num_docs=len(corpus), num_terms=lda_model.num_topics, dtype = np.float32)
    theta = np.asarray(theta, dtype = np.float32)
    theta = theta.transpose()
    best_score = 0
    best_dist = None
    for (dist, f) in dist_fns.items():
        score = average_precision_at_k(theta, labels, 100, f)
        if score > best_score:
            best_score, best_dist = score, dist
    return best_score, best_dist

In [65]:
for num_topics in range(5, 60 + 1,5):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    score, dist = eval_topic_model(lda_model, df.target)
    print("%2i topics AP@100 = %.3f (%s distance)" % (num_topics, score, dist))

D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


 5 topics AP@100 = 0.586 (dot distance)


D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


10 topics AP@100 = 0.504 (eucl distance)


D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


...

50 topics AP@100 = 0.401 (eucl distance)


D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


55 topics AP@100 = 0.460 (cosine distance)


D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


60 topics AP@100 = 0.407 (dot distance)


In [66]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

D:\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
28     0.360870  0.195106       1        1  24.196138
29     0.331148  0.182178       2        1  15.098852
39     0.301168 -0.021882       3        1   9.654282
20     0.262202 -0.171326       4        1   8.214921
24     0.283331  0.187367       5        1   4.762838
52    -0.134930  0.033403       6        1   3.624466
45     0.178909 -0.101053       7        1   2.379766
26     0.179042  0.157626       8        1   1.848481
21     0.137571 -0.044416       9        1   1.818102
7      0.123965 -0.171840      10        1   1.696884
42     0.144588  0.039439      11        1   1.475969
55     0.145528  0.095697      12        1   1.429484
59     0.086679 -0.105520      13        1   1.312353
34     0.074586 -0.077589      14        1   1.220881
6      0.097626 -0.083809      15        1   1.192170
58     0.004881 -0.005582      16        1   1.009104
51     0.056882 -0.035245      17        1   0.968312
49     0.038691 -0.048344      18        1   0.902002
44     0.054804 -0.019446      19        1   0.899856
11     0.039165 -0.093533      20        1   0.871156
10     0.049983 -0.067676      21        1   0.828803
36     0.060520 -0.079663      22        1   0.807685
17     0.037185 -0.039889      23        1   0.787934
14     0.055111 -0.113070      24        1   0.752631
48     0.037170 -0.029992      25        1   0.685880
46    -0.008094 -0.031018      26        1   0.637178
13    -0.014475 -0.034576      27        1   0.628601
37    -0.014856 -0.018394      28        1   0.574192
50    -0.017275 -0.035028      29        1   0.571886
9     -0.038504  0.003993      30        1   0.535607
22    -0.015620 -0.018277      31        1   0.534203
57    -0.028625 -0.006563      32        1   0.525415
41    -0.019059 -0.017560      33        1   0.501300
56    -0.024263 -0.011447      34        1   0.491605
15    -0.029503 -0.000296      35        1   0.480343
3     -0.018893 -0.008661      36        1   0.463439
38    -0.059157  0.006095      37        1   0.447411
47    -0.003366  0.043419      38        1   0.435612
12    -0.051107  0.001595      39        1   0.435012
32    -0.024584 -0.012808      40        1   0.414402
54    -0.060990  0.008936      41        1   0.380441
19    -0.054502  0.003710      42        1   0.379947
4     -0.066963  0.011603      43        1   0.377769
5     -0.088732  0.018951      44        1   0.356224
2     -0.092050  0.029905      45        1   0.293216
40    -0.093637  0.020471      46        1   0.292903
25    -0.118918  0.026537      47        1   0.277821
33    -0.080225  0.020627      48        1   0.270685
23    -0.098773  0.022399      49        1   0.240849
1     -0.121151  0.029749      50        1   0.237955
30    -0.116909  0.025711      51        1   0.229485
53    -0.140499  0.033038      52        1   0.161091
16    -0.162863  0.038659      53        1   0.132936
0     -0.170852  0.039449      54        1   0.102591
35    -0.185283  0.039712      55        1   0.057648
18    -0.188655  0.039490      56        1   0.049235
27    -0.192963  0.038987      57        1   0.032014
8     -0.201378  0.037021      58        1   0.006370
43    -0.201977  0.036815      59        1   0.002851
31    -0.201977  0.036815      60        1   0.002817, topic_info=      Category          Freq               Term         Total  loglift  \
term                                                                     
2450   Default  30588.000000                max  30588.000000  30.0000   
21     Default  18438.000000               line  18438.000000  29.0000   
30     Default  16918.000000       organization  16918.000000  28.0000   
1120   Default   7602.000000                  _   7602.000000  27.0000   
245    Default  14226.000000              write  14226.000000  26.0000   
55     Default  10773.000000            article  10773.000000  25.0000   
82     Default  2

In [67]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -28.529014837842524

Coherence Score:  0.4556655591465935
